This notebook is used to use line-plot and ked-plot to demonstrate how neural activity varies with time in two different orthogonal conditions. 

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.io as scio
import pandas as pd
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d

# Load data

In [2]:
data = scio.loadmat('dpca_results.mat')['dpca_results']
print(f"data is a {data.shape} size ndarray, with each element is a struct array")
print(f"Fieldnames of each element are {data.dtype.names}")

data is a (1, 10) size ndarray, with each element is a struct array
Fieldnames of each element are ('name', 'Z', 'W', 'explVar', 'whichMarg')


# Positive example

This section is used to plot left panel of (d).

Here we only select trials in 20211115_ab to display. 

In [3]:
day = 8
dname, Z, whichMarg = str(data[0, day]['name'][0]), data[0, day]['Z'], data[0, day]['whichMarg']
dname, type(Z), type(dname), type(whichMarg), Z.shape

('20211215_ab_cl_new_all_firingrate_001.mat',
 numpy.ndarray,
 str,
 numpy.ndarray,
 (20, 4, 2, 2000))

Below we define some useful plot functions for traj plot.

In [4]:
def Z_to_df(Z, axis_list=[1, 2, 3]):
    '''
    Converts Z to dataframe for sns plot
    :param Z: ndarray. n_comps * n_stim * n_mot * n_tim
    :param axis_list: [stim1_comp, stim2_comp, mot_comp]
    :return: df with columns [stim1_comp, stim2_comp, mot_comp, stim_label, mot_label]
    '''
    _, n_stim, n_mot, n_time = Z.shape
    assert len(axis_list) == 3, "Expecting 3 elements in arg 1"
    stim_label, mot_label = (np.zeros((3, n_stim, n_mot, n_time), dtype=np.uint8),
                             np.zeros((3, n_stim, n_mot, n_time), dtype=np.uint8))
    for i in range(n_stim):
        stim_label[:, i, :, :] = stim_label[:, i, :, :] + i + 1
    for i in range(n_mot):
        mot_label[:, :, i, :] = mot_label[:, :, i, :] + i + 1
    df = pd.DataFrame({
        'stimulus1': np.ravel(Z[axis_list[0], :, :, :]),
        'stimulus2': np.ravel(Z[axis_list[1], :, :, :]),
        'motion': np.ravel(Z[axis_list[2], :, :, :]),
        'stim_label': np.ravel(stim_label[0, :, :, :]),
        'mot_label': np.ravel(mot_label[0, :, :, :])
    })
    return df

def get_traj_df(Z, whichMarg, axis_list=None):
    
    if axis_list is None:
        stim_idx = np.where(whichMarg == 1)[1]
        mot_idx = np.where(whichMarg == 2)[1]
        axis_list = np.hstack((stim_idx[:2], mot_idx[0]))
    df = Z_to_df(Z, axis_list=axis_list)
    return df

def plot_traj_intensity(df, palette, x='stimulus1', y='stimulus2', hue='stim_label', 
                        xlim=None, ylim=None, n_ticks=6):
    fig = plt.figure(figsize=(8, 6))
    sns.kdeplot(df, x=x, y=y, palette=palette, 
                    hue=hue, fill=True, alpha=0.3, cmap='mako')
    ax = plt.gca()
    # Tighten the x-axis and y-axis limits to the data extent
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:
        ax.set_ylim(ylim)
    
    # Set color of x-axis line to black
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    # set width
    ax.spines['bottom'].set_linewidth(1.3)
    ax.spines['left'].set_linewidth(1.3)
    # set box off
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    # display less ticks
    plt.locator_params(axis='both', nbins=n_ticks)
    plt.tight_layout()
    
    return fig

In [5]:
# cfg
linewidth = 3
fontsize = 6
title_fontsize = 7
plt.rcParams.update({'font.size': fontsize})
figsize = (12, 8)

def traj_plot(dname, Z, whichMarg, over_time, palette, downsample_factor=5, 
              axis_list=None, n_ticks=4, set_label=True, limits=None):
    linestyles = ['solid', 'dashed', 'dashdot']
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(projection='3d')
    ax.set_facecolor('white')
    # Set the color of the axis lines to k
    ax.xaxis.line.set_color('k')
    ax.yaxis.line.set_color('k')
    ax.zaxis.line.set_color('k')
    
    # Set the color of the pane to white
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    
    # Now set color to white (or whatever is "invisible")
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    
    # display less ticks
    plt.locator_params(axis='both', nbins=n_ticks)
    
    # Set display rotation angles (azimuthal and elevation)
    ax.view_init(azim=30, elev=15)
    
    over_time = np.arange(start=over_time[0], stop=over_time[1], step=downsample_factor)
    stim_idx = np.where(whichMarg == 1)[1]
    mot_idx = np.where(whichMarg == 2)[1]
    if axis_list is None:
        axis_list = np.hstack((stim_idx[:2], mot_idx[0]))
    elif not isinstance(axis_list, np.ndarray):
        axis_list = np.array(axis_list)
    for i in range(Z.shape[1]):
        for j in range(Z.shape[2]):
            axis_plot = []
            for k in range(len(axis_list)):
                axis_plot.append(gaussian_filter1d(Z[axis_list[k], i, j, over_time], sigma=5, axis=-1))
            ax.plot(axis_plot[0], axis_plot[1], axis_plot[2], color=palette[i], linestyle=linestyles[j], linewidth=linewidth)
            # start point
            ax.scatter(axis_plot[0][0], axis_plot[1][0], axis_plot[2][0], color=palette[i], marker='o')
    
    if set_label:
        ax.set_xlabel('stimulus 1', labelpad=1)
        ax.set_ylabel('stimulus 2', labelpad=1)
        ax.set_zlabel('motion', labelpad=1)
        
    # axis limits
    e = 1e-5
    if limits is not None:
        ax.axes.set_xlim3d(left=limits[0][0]+e, right=limits[0][1]-e)
        ax.axes.set_ylim3d(bottom=limits[1][0]+e, top=limits[1][1]-e)
        ax.axes.set_zlim3d(bottom=limits[2][0]+e, top=limits[2][1]-e)
    
    # axis padding
    ax.tick_params(axis='x', which='major', pad=1)
    ax.tick_params(axis='y', which='major', pad=1)
    ax.tick_params(axis='z', which='major', pad=1)
    
    # ax.grid(False)
    plt.tight_layout()
    plt.show()
    
    return axis_list, fig

Smooth data.

In [6]:
ws = 10
poly_order = 3
Z = savgol_filter(Z, ws, poly_order, axis=-1)

Plot setting:

In [7]:
palette = ['#0a3e78', '#d3a972', '#AB2328', '#5c9167']
axis_list = None
image_save_dpi = 500
### save cfg ####
ifsave = False
# save folder
image_folder = "final"
# save format
image_format = "tif"
# save date
date = dname.split('_')[0][4:]
# save monkey
monkey = dname.split('_')[1]

Plot traj and intensity in the same figure:

In [8]:
# plot traj and intensity
# 1215
limits = [
    [-10, 10],
    [-10, 10],
    [-10, 15]
]
_, fig = traj_plot(dname, Z, whichMarg, palette=palette, over_time=[0, 1500], limits=limits,
          downsample_factor=50, axis_list=axis_list, n_ticks=5, set_label=True)

# add kde
df = get_traj_df(Z, whichMarg, axis_list=axis_list)
sns.kdeplot(df, x='stimulus1', y='stimulus2', palette=palette, 
                hue="stim_label", fill=True, alpha=0.3, cmap='mako')

# save traj and kde
if ifsave:
    image_name = f"./{image_folder}/{date}_{monkey}_traj_intense.tif"
    image_format = "tif"
    fig.savefig(image_name, format=image_format, dpi=image_save_dpi)

/Users/gelingling/miniconda3/envs/BrainCog/lib/python3.8/site-packages/seaborn/distributions.py:1128: UserWarning: cmap parameter ignored when using hue mapping.
  warnings.warn(msg, UserWarning)


Plot traj only:

In [9]:
# plot traj only
limits = [
    [-10, 10],
    [-10, 10],
    [-10, 15]
]
_, fig = traj_plot(dname, Z, whichMarg, palette=palette, over_time=[0, 1500], limits=limits,
          downsample_factor=50, axis_list=axis_list, n_ticks=5, set_label=True)

if ifsave:
    image_name = f"./{image_folder}/{date}_{monkey}_traj.tif"
    fig.savefig(image_name, format=image_format, dpi=image_save_dpi)

Plot intensity only:

In [10]:
# plot intensity only
limits = [
    [-12, 12],
    [-15, 15]
]

df = get_traj_df(Z, whichMarg, axis_list=axis_list)
fig2 = plot_traj_intensity(df, palette, x='stimulus1', y='stimulus2', 
                        hue="stim_label", xlim=limits[0], ylim=limits[1], n_ticks=6)

if ifsave:
    image_name = f"./{image_folder}/{date}_{monkey}_intense.tif"
    image_format = "tif"
    fig2.savefig(image_name, format=image_format, dpi=image_save_dpi)

/Users/gelingling/miniconda3/envs/BrainCog/lib/python3.8/site-packages/seaborn/distributions.py:1128: UserWarning: cmap parameter ignored when using hue mapping.
  warnings.warn(msg, UserWarning)


# Negative example

This section is used to plot right panel of (d).

Here we only select trials in 20211208_ab to display. 

In [11]:
day = 6
dname, Z, whichMarg = str(data[0, day]['name'][0]), data[0, day]['Z'], data[0, day]['whichMarg']
dname, type(Z), type(dname), type(whichMarg), Z.shape

('20211208_ab_cl_new_all_firingrate_001.mat',
 numpy.ndarray,
 str,
 numpy.ndarray,
 (20, 4, 2, 2000))

In [12]:
ws = 10
poly_order = 3
Z = savgol_filter(Z, ws, poly_order, axis=-1)

In [13]:
palette = ['#0a3e78', '#d3a972', '#AB2328', '#5c9167']
axis_list = [15, 16, 12]
image_save_dpi = 500
### save cfg ####
ifsave = False
# save folder
image_folder = "final"
# save format
image_format = "tif"
# save date
date = dname.split('_')[0][4:]
# save monkey
monkey = dname.split('_')[1]

In [14]:
# plot traj and intensity
limits = [
    [-3, 3],
    [-4, 5],
    [-3, 4]
]
_, fig = traj_plot(dname, Z, whichMarg, palette=palette, over_time=[0, 1500], limits=limits,
          downsample_factor=50, axis_list=axis_list, n_ticks=5, set_label=True)

# add kde
df = get_traj_df(Z, whichMarg, axis_list=axis_list)
sns.kdeplot(df, x='stimulus1', y='stimulus2', palette=palette, 
                hue="stim_label", fill=True, alpha=0.3, cmap='mako')

# save traj and kde
if ifsave:
    image_name = f"./{image_folder}/{date}_{monkey}_traj_intense.tif"
    image_format = "tif"
    fig.savefig(image_name, format=image_format, dpi=image_save_dpi)

/Users/gelingling/miniconda3/envs/BrainCog/lib/python3.8/site-packages/seaborn/distributions.py:1128: UserWarning: cmap parameter ignored when using hue mapping.
  warnings.warn(msg, UserWarning)


In [15]:
# plot traj only
limits = [
    [-0.5, 1.5],
    [-1, 2],
    [-1, 1.5]
]
_, fig = traj_plot(dname, Z, whichMarg, palette=palette, over_time=[0, 1500], limits=limits,
          downsample_factor=50, axis_list=axis_list, n_ticks=5, set_label=True)

if ifsave:
    image_name = f"./{image_folder}/{date}_{monkey}_traj.tif"
    fig.savefig(image_name, format=image_format, dpi=image_save_dpi)

In [16]:
# plot intensity only
limits = [
    [-5, 5],
    [-7, 6]
]
df = get_traj_df(Z, whichMarg, axis_list=axis_list)
fig2 = plot_traj_intensity(df, palette, x='stimulus1', y='stimulus2', 
                        hue="stim_label", xlim=limits[0], ylim=limits[1], n_ticks=6)

if ifsave:
    image_name = f"./{image_folder}/{date}_{monkey}_intense.tif"
    image_format = "tif"
    fig2.savefig(image_name, format=image_format, dpi=image_save_dpi)

/Users/gelingling/miniconda3/envs/BrainCog/lib/python3.8/site-packages/seaborn/distributions.py:1128: UserWarning: cmap parameter ignored when using hue mapping.
  warnings.warn(msg, UserWarning)


In [17]:
day = 8
dname, Z, whichMarg = str(data[0, day]['name'][0]), data[0, day]['Z'], data[0, day]['whichMarg']
dname, type(Z), type(dname), type(whichMarg), Z.shape

('20211215_ab_cl_new_all_firingrate_001.mat',
 numpy.ndarray,
 str,
 numpy.ndarray,
 (20, 4, 2, 2000))